In [3]:
import os

import bk.load
import bk.compute
import bk.plot
import bk.signal

import numpy as np
import pandas as pd

import scipy.signal
import scipy.stats
import scipy.io

import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts


import itertools
from tqdm import tqdm



In [4]:
def training_intervals():
    run_ = scipy.io.loadmat('runintervals.mat')['runintervals']
    runintervals = nts.IntervalSet(run_[:,0],run_[:,1],time_units='s')
    return runintervals

In [56]:
def mean_firing(neurons,intervals):
    binSize = 0.5
    t,fr = bk.compute.binSpikes(neurons,binSize)
    t = nts.Ts(t,time_units='s')
    fr = np.mean(fr[:,intervals.in_interval(t)>= 0]/binSize,0)
    t = t[intervals.in_interval(t)>=0]
    fr = nts.Tsd(t.index.values, d = fr)
    fr = bk.compute.nts_smooth(fr,1000,1)
    return fr

In [57]:
COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['figure.facecolor'] = '#282C34'
mpl.rcParams['axes.facecolor'] = '#282C34'

In [58]:
path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
os.chdir(path)
bk.load.path = path
bk.load.rat = 8
bk.load.session = 'Rat08-20130713'
bk.load.day = 6

In [59]:
# bk.load.current_session()
neurons,metadata = bk.load.loadSpikeData(bk.load.path)
states = bk.load.states()
pos = bk.load.pos()
train = training_intervals()
pos = pos.restrict(train)
speed = bk.compute.old_speed(pos,1)
speed = speed[speed.values>1]

Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [60]:
import sys
eps = sys.float_info.epsilon

In [74]:
%matplotlib qt
plt.figaspect(4/16)
stru = ['Hpc','BLA']
types = ['Pyr','Int']
plt.subplot(1,3,1)
i = 1
for s,t in itertools.product(stru,types):
    print(s,t)
    neur = neurons[(metadata.Region == s) & (metadata.Type == t)]
    meta = metadata[(metadata.Region == s) & (metadata.Type == t)]
    for n,shank,id in zip(neur,meta['Shank'],meta['Id']):
        fr = mean_firing([n],train)
        stats = (scipy.stats.linregress(speed.realign(fr).values,np.log(fr.values+eps)))
        if (stats.pvalue <0.05) & (stats.rvalue > 0.2):
            print(stats)
            plt.subplot(1,3,i)
#             plt.figure()
            plt.hist2d(speed.realign(fr).values,np.log(fr.values+eps),50)
            x = np.linspace(0,5,2500)
            y = stats.intercept + x*stats.slope
            plt.plot(x,y,'red')
            plt.title(s+t+' '+str(shank)+' '+str(id))
            print(scipy.stats.linregress(speed.realign(fr).values,fr.values+eps))
            print(np.corrcoef(speed.realign(fr),fr))
            plt.ylabel('FiringRates')
            plt.xlabel('Speed(cm/s)')
            i+=1
# plt.legend(list(itertools.product(stru,types)))

# plt.subplot(2,1,2,sharex = sub)
# plt.plot(speed)

Hpc Pyr
LinregressResult(slope=0.5640242020104995, intercept=-31.37871454495038, rvalue=0.21319022352471956, pvalue=1.2155970519502652e-76, stderr=0.030086535208783256)
LinregressResult(slope=0.014872299422137105, intercept=0.04458245037287928, rvalue=0.13841014694322354, pvalue=6.623379441913047e-33, stderr=0.001238660404306618)
[[1.         0.13841015]
 [0.13841015 1.        ]]


/home/billel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  app.launch_new_instance()


Hpc Int
LinregressResult(slope=0.025999515753573055, intercept=2.787721083902629, rvalue=0.32935601372438733, pvalue=1.8557903628379554e-187, stderr=0.0008648874975719297)
LinregressResult(slope=0.5383110028556509, intercept=17.176791606141105, rvalue=0.3686043804849491, pvalue=7.718345378674901e-238, stderr=0.015752728857256666)
[[1.         0.36860438]
 [0.36860438 1.        ]]
BLA Pyr
BLA Int
LinregressResult(slope=0.037885700408637556, intercept=3.0151145584433134, rvalue=0.42560688425258697, pvalue=0.0, stderr=0.0009346832706645284)
LinregressResult(slope=1.3447520967570383, intercept=20.674661730394856, rvalue=0.531203102079887, pvalue=0.0, stderr=0.024887637955123623)
[[1.        0.5312031]
 [0.5312031 1.       ]]


In [16]:
bk.load.current_session('Z:\\Rat08\\Rat08-20130710')
neurons,metadata = bk.load.loadSpikeData(bk.load.path)

for i in np.unique(metadata['Shank']):
    print(i,len(neurons[metadata['Shank']==i]))

FileNotFoundError: [Errno 2] No such file or directory: 'Z:/All-Rats/Billel/session_indexing.csv'

In [287]:
a = np.array([np.nan])

AttributeError: 'numpy.ndarray' object has no attribute 'dropna'